In [ ]:
%pip install h5py numpy xarray netcdf4

In [2]:
import numpy as np
import h5py
import xarray as xr

## Writing Mixed-Type, Labelled, Data and Metadata to HDF5 Files using `h5py`

HDF5 files have a lot more features:
  - They are highly cross-platform and work with a wide variety of tools
  - They can store many different datasets in a single file (or even in multiple linked files)
  - They can store metadata alonside the data
  - They let you store data hierarchically, making a nice dict-like nested organization for your data
  - They can compress your data.
  - They let you work with data that is larger than memory, making it easy to read in only the data that you need.
  - They can be easily previewed and inspected using the https://myhdf5.hdfgroup.org/ web tool!
  
So many features!  Here, we'll get a basic senses of how they work by using the `h5py` library, which gives us a dict-like, Numpy-native interface to HDF5 files and is used internally by many popular Python frameworks.



| Code | Description |
| :-- | :-- |
| **`f = h5py.File('filename.h5', 'w')`** | Open an h5py file object for writing |
| **`f.close()`** | Closes the h5py file and releases the linked file back to the operating system. |
| **`f.create_dataset('temp', data=x)`** | Write an array called 'temp' with the data in the numy array `x` into the HDF5 file |
| **`f.create_dataset('data/temp', data=x)`** | Write an array called 'temp' in the folder called "data" with the data in the numy array `x` into the HDF5 file |
| **`f.attrs['name'] = 'Session 1'`** | Set an attribute as metadata onto the root group of the HDF5 file -- this works like a normal Python dictionary |
| **`f['x'].attrs['id'] = 'ABC'`** | Set an attribute as metadata onto the 'x' node of the HDF file |



--- 

### Storing Arrays in HDF5 as "Datasets" and Organizing them in "Groups"

| Code | Description |
| :-- | :-- |
| **`f = h5py.File('filename.h5', 'w')`** | Open an h5py file object for writing |
| **`f.close()`** | Closes the h5py file and releases the linked file back to the operating system. |
| **`f['x'] = np.array([1, 2, 3])`** | Put a Numpy array into a **dataset** called `x` |
| **`f['folder/x'] = np.array([1, 2, 3])`** | Put a Numpy array into  **dataset** called `x`, in a **group** called `folder` |
| **`f.create_dataset('folder/x', shape=(100,2), dtype=np.uint8)`** | Make an empty dataset in the file |
| **`f.create_dataset('folder/x', data=my_array, compression='gzip')`** | Store a numpy array as a dataset using a given compression algorithm. |
| **`f['x'][:]`** | Read in the dataset into a Numpy array. |

**Exercises**

**Exercise: Open and Close a File**.  Open an HDF5 file named `exercise1.h5` for storing EEG data, then close it.


In [8]:
with h5py.File('exercise1.h5', 'w') as f:
    f


**Exercise: Store a Small Array of Spike Counts**. Store a small 1D NumPy array of spike counts in a file called `exercise2.h5`, in a dataset called `spike_counts`.


In [9]:
with  h5py.File('exercise2b.h5', 'w') as f:
    f['spikes_counts'] = np.array([1, 3, 5, 9])

**Exercise: Read Back the Spike Counts**.  Read the `spike_counts` dataset from `exercise2.h5`.

In [11]:
with  h5py.File('exercise2b.h5') as f:
    data = f['spikes_counts'][:]
data

array([1, 3, 5, 9])

**Exercise: Create a Dataset Inside a Group for LFP Data**. Create a dataset named `lfp_data` inside a group called `session1`.

In [12]:
with h5py.File('exercise3.h5', 'w') as f:
    f['session1/lfp_data'] = np.array([1, 2, 3])

**Exercise: Read the LFP Dataset from the Folder**.  Read the `lfp_data` from the `session1` group in the previous exercise.


In [13]:
with h5py.File('exercise3.h5') as f:
    data = f['session1/lfp_data'][:]
data


array([1, 2, 3])

**Exercise: Create an Empty Dataset for Neuron Responses**. Create an empty dataset called `neuron_responses` with shape (100, 2).


In [16]:
with h5py.File('exercise4.h5', 'w') as f:
    f.create_dataset('neuron_responses', shape=(100, 2))

**Exercise: Compressed Dataset for EEG**.  Create a compressed dataset called `eeg_data` using gzip.

**Exercise: Read the Compressed EEG Data**. Read the `eeg_data` dataset from the previous exercise. 

**Exercise: Put Multiple Datasets into a Schema**:  Create a file with the following organization:

  - behavior/
    - trial: 10 values, uint8
    - time: 100 values,  float32 
    - lick_rate: 10 x 100 values, float32
  - ephys/
    - spike_times: 50 values, float64


### Storing Metadata in HDF5 as "Attributes"

Labeling our data and keeping those labels together with the data itself is a key practice in having well-organized data, and hdf5 makes it easy to put labels directly inside the files.  

One practice to be aware of:  
  - General metadata for the file is often attached to the "root" group, so it's easy to find.  Things like experiment-level or session-level metadata are often stored there.  
  - Metadata that describes a specific array are often attached to the hdf5 dataset itself.  Things like dimension labels, units, text descriptions of what the data represents can usually be found here.

.

| Code | Description |
| :-- | :-- |
| **`f.attrs['subject'] = 'Doug'`** | Add a "subject" attribute to the root-level group. |
| **`f['x'].attrs['subject'] = 'Doug'`** | Add a "subject" attribute to the 'x' group or dataset. |
| **`dict(f.attrs)`** | Get all the attributes attached to the root-level group as a dict.
| **`dict(f['x'].attrs)`** | Get all the attributes attached to the 'x' group or dataset as a dict.


**Exercises**

**Exercise**: Create a file with the following schema:

```
meta_ex1.h5
└─ / (root)
   └─ subject_id = "RatA"  (attribute)
```



**Exercise**: Reopen `meta_ex1.h5` and get the "subject_id" attribute value from the file.

**Exercise**: Create a file with the following schema:
```
meta_ex2.h5
└─ / (root)
   └─ lfp_data (dataset, shape=(100,))
      └─ recording_date = "2025-03-31"  (attribute)
```

**Exercise:** Reopen `meta_ex2.h5` and the `recording_date` attribute.

**Exercise**: Create a file with the following schema:

```
meta_ex3.h5
└─ / (root)
   ├─ experimenter = "Dr. Gray"       (attribute)
   ├─ experiment_type = "Optogenetics" (attribute)
   └─ lab = "NeuroLab"                (attribute)
   └─ spike_trains (dataset, shape=(4,50))
      ├─ brain_region = "Hippocampus"   (attribute)
      └─ num_channels = 4              (attribute)
```



**Exercise:** Reopen `meta_ex3.h5` and get all the root-level metadata into a dict.

--- 

### Labeling Dimensions of Datasets by Linking Arrays

| Code | Description |
| :-- | :-- |
| **`f['temperature'].dims[0].label = 'time'`** | Mark the first dimension of the `temperature` dataset as going over time. |
| **`f['time'].make_scale('Time')`** | Label the `time` dataset as a dimension scale with name "Time". |
| **`f['temperature'].dims[0].attach_scale(f['time'])`** | Link each value of `time` to each value of `temperature`. |
| **`xr.load_dataset('myfile.h5')`** | XArray will recognize and organize linked datasets automatically. |

Reference: https://docs.h5py.org/en/stable/high/dims.html


#### Example: Single-Dimension LFP Data Over Time
- **Goal**: Create a file named `ex1.h5` with:
  - A 1D dataset called `lfp_data` (e.g., shape `(1000,)`).
  - A 1D dataset called `time` (same length, storing time points).
  - Make `time` a dimension scale named `"Time"`.
  - Attach this scale to the first (and only) dimension of `lfp_data`.
  - Label that dimension `"time"`.
- **Check**: Call `xr.load_dataset('ex1.h5')` and confirm it recognizes `lfp_data` with a coordinate named `"time"`.


In [ ]:
import h5py
import numpy as np
import xarray as xr


with h5py.File('ex1.h5', 'w') as f:
    f['time'] = np.linspace(0, 10, 1000)  # 1000 data points spanning 0 to 10 seconds
    f['time'].make_scale('Time')  # Mark the 'time' dataset as a dimension scale named "Time"

    f['lfp_data'] = np.random.randn(1000) # 1000 normally-distributed random values.
    f['lfp_data'].dims[0].label = 'time'  # # Label the first dimension as "time"
    f['lfp_data'].dims[0].attach_scale(f['time'])

xr.load_dataset('ex1.h5')


<xarray.Dataset> Size: 16kB
Dimensions:   (time: 1000)
Coordinates:
  * time      (time) float64 8kB 0.0 0.01001 0.02002 0.03003 ... 9.98 9.99 10.0
Data variables:
    lfp_data  (time) float64 8kB 0.852 1.118 2.278 ... 1.416 0.23 -0.6421

#### Exercise 2: Two-Dimensional LFP Data (Time × Channels)
- **Goal**: Create a file named `ex2.h5` with:
  - A 1D dataset `time` (shape `(1000,)`) representing time points.
  - A 1D dataset `channels` (shape `(16,)`) representing electrode channels.
  - A 2D dataset `lfp_data` (shape `(1000, 16)`).
  - Make `time` and `channels` dimension scales (`"Time"` and `"Channels"`).
  - Attach them as dimension scales to `lfp_data` on dim 0 and dim 1, respectively.
  - Label these dimensions `"time"` and `"channel"`.
- **Check**: `xr.load_dataset('ex2.h5')` should show `lfp_data` with coordinates `("time", "channel")`.




#### Exercise 3: ROI Signals (Time × ROI)
- **Goal**: Create `ex3.h5` with:
  - A 1D dataset `time` (e.g., `(1200,)`) for frames in a calcium imaging experiment.
  - A 1D dataset `roi` (e.g., `(5,)`) for different regions of interest.
  - A 2D dataset `roi_signals` (shape `(1200, 5)`).
  - Make and attach scales with names `"Time"` for `time` and `"ROI"` for `roi`.
  - Label dimensions as `"time"` and `"roi"`.
- **Check**: `xr.load_dataset('ex3.h5')` should yield `roi_signals` with xarray dims `["time", "roi"]`.


#### Exercise 4: Imaging Data (Time × X × Y)
- **Goal**: Create `ex6.h5` with:
  - A 1D dataset `time` (e.g., `(50,)`) for imaging frames.
  - A 1D dataset `x` (e.g., `(64,)`) for X-coordinates in the image.
  - A 1D dataset `y` (e.g., `(64,)`) for Y-coordinates in the image.
  - A 3D dataset `image_stack` (shape `(50, 64, 64)`) for pixel intensities.
  - Make `time`, `x`, and `y` dimension scales named `"Time"`, `"X"`, `"Y"`.
  - Attach them to the corresponding dimensions of `image_stack`.
  - Label these dimensions `"time"`, `"x"`, and `"y"`.
- **Check**: `xr.load_dataset('ex6.h5')` should load `image_stack` with three named dimensions.
